# Введение в обработку естественного языка
## Урок 14. Transfer learning
Задание 14
Задание взять данные из https://www.kaggle.com/datasets/mrapplexz/bashim-quotes обучить модель GPT для генерации своих цитат
взять новостные данные из https://github.com/natasha/corus load_lenta2 нам понадобиться сам текст и заголовок обучить модель T5/ или GPT для генерации заголовков для статей

**Генерация цитат**

In [7]:
import pandas as pd
import json
import re
!pip install transformers
!pip install datasets
pd.set_option('max_colwidth', 400)
import re
from sklearn.model_selection import train_test_split
import numpy as np
from datasets import Dataset
from transformers import AutoTokenizer
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import T5ForConditionalGeneration, Trainer, TrainingArguments
import gc
import torch

In [9]:
data = pd.read_json('dataset.jsonl', lines=True)
data.head()

,id,date,rating,text
0,1,2004-08-30 11:24:00+00:00,22010.0,"<Ares> ppdv, все юниксы очень дружелюбны.. они просто очень разборчивы в друзьях ;)"
1,2,2004-08-30 11:25:00+00:00,25105.0,"<томатик_рад> а ты не чувствуешь красоту мира?\n<fox> честно говоря, я сейчас чувствую только отсутствие http.\n<томатик_рад> не туда смотришь, глянь вокруг!\n<fox> как я гляну, если http не работает? :/"
2,3,2004-08-30 11:27:00+00:00,7192.0,"<Дор> ""мышка, почему у тебя такие большие глаза?"" УЙДИ!!! я ХАРАКИРИ делаю!!!!!!"
3,4,2004-08-30 11:28:00+00:00,29169.0,"<PPDV[os2]> ""Мальчики, вы что больные, бегать в палату к девочкам?! - Если б мы были больные - мы б бегали к другим мальчикам"""
4,5,2004-08-30 11:26:00+00:00,7140.0,<Ohtori_Akio> мы - как разработчики - живём с субейзом под одбц. \n<Ohtori_Akio> лучше бы мы жили в пещере с гоблинами.


In [10]:
data.shape

(81497, 4)

In [11]:
# Функция создания текстового файла с предобработкой
# Уберем никнеймы из цитат

def build_text_files(data_json, dest_path):
    f = open(dest_path, 'w')
    data = ''
    for texts in data_json:
        summary = texts.strip()
        summary = re.sub(r'<.*?>', '', summary)
        data += summary + ' '
    f.write(data)

In [12]:
train, test = train_test_split(data, test_size=0.3)

build_text_files(train,'train_dataset.txt')
build_text_files(test,'test_dataset.txt')

In [13]:
print('Train dataset length: ' + str(len(train)))
print('Test dataset length: ' + str(len(test)))

Train dataset length: 57047
Test dataset length: 24450


In [ ]:
from transformers import AutoTokenizer
from transformers import Trainer, TrainingArguments, AutoModelForCausalLM

#sberbank-ai/rugpt3large_based_on_gpt2
#sberbank-ai/rugpt3medium_based_on_gpt2
#sberbank-ai/rugpt3small_based_on_gpt2

tokenizer = AutoTokenizer.from_pretrained('sberbank-ai/rugpt3medium_based_on_gpt2')
model = AutoModelForCausalLM.from_pretrained('sberbank-ai/rugpt3medium_based_on_gpt2')

train_path = 'train_dataset.txt'
test_path = 'test_dataset.txt'

In [ ]:
from transformers import TextDataset, DataCollatorForLanguageModeling

def load_dataset(train_path, test_path, tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)

    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset, test_dataset, data_collator

train_dataset, test_dataset, data_collator = load_dataset(train_path, test_path, tokenizer)

In [ ]:
tokenizer.decode(train_dataset[2])

In [ ]:
training_args = TrainingArguments(
    output_dir='./gpt2-chief', 
    overwrite_output_dir=True,
    num_train_epochs=10, 
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4, 
    eval_steps=100, # Number of update steps between two evaluations.
    save_steps=400, # after # steps model is saved
    warmup_steps=500, # number of warmup steps for learning rate scheduler
    optim = 'adamw_torch',
    report_to='all'
    )

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

trainer.train()

In [ ]:
trainer.save_model()
tokenizer.save_pretrained('gpt_quotes')
model.save_pretrained('model_gpt_quotes')

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('/content/drive/MyDrive/Colab Notebooks/NLP/gpt_quotes')
model_new = AutoModelForCausalLM.from_pretrained('/content/drive/MyDrive/Colab Notebooks/NLP/model_gpt_quotes').to('cuda') 

In [ ]:
def generate_quotes(prefix):
    tokens = tokenizer(prefix, return_tensors='pt').to('cuda')
    size = tokens['input_ids'].shape[1]
    output = model_new.generate(
        **tokens, 
        pad_token_id=tokenizer.eos_token_id,
        do_sample=False,
        max_length=size + 50, 
        repetition_penalty=5., 
        # temperature=5,
        num_beams=80,
        # early_stopping=True,
        no_repeat_ngram_size=1)

    decoded = tokenizer.decode(output[0])
    result = decoded[len(prefix):]
    print(f'{prefix} {result}\n *******\n')

In [ ]:
generate_quotes('Эээ')
generate_quotes('Ну что ж')
generate_quotes('Сегодня утром')
generate_quotes('Я хочу сказать, что')
generate_quotes('Кручу верчу')
generate_quotes('Не надо изысканности')
generate_quotes('О!')

 Чем больше num_beams, тем лучше генерируется текст

**Генерация заголовков статей**

In [ ]:
from corus import load_lenta

path = 'lenta-ru-news.csv.gz'
records = load_lenta(path)
next(records)

LentaRecord(
    url='https://lenta.ru/news/2018/12/14/cancer/',
    title='Названы регионы России с\xa0самой высокой смертностью от\xa0рака',
    text='Вице-премьер по социальным вопросам Татьяна Голикова рассказала, в каких регионах России зафиксирована наиболее высокая смертность от рака, сооб...',
    topic='Россия',
    tags='Общество'
)

In [ ]:
data_lenta = pd.read_csv('lenta-ru-news.csv')
data_lenta.head()

In [ ]:
data_lenta.info()

In [ ]:
data_lenta.dropna(subset=['text'], inplace=True)
data_lenta.reset_index(inplace=True)

In [ ]:
train, test = train_test_split(data_lenta[:10000], test_size=0.3)

In [ ]:
print('Train dataset length: ' + str(len(train)))
print('Test dataset length: ' + str(len(test)))

In [ ]:
max_len_sum = int(np.mean([len(data_lenta['text'][txt]) for txt in range(len(data_lenta))])/2.5)
max_len_tl = int(np.mean([len(data_lenta['title'][txt]) for txt in range(len(data_lenta))]))

In [ ]:
dataset_train = Dataset.from_pandas(train)
dataset_test = Dataset.from_pandas(test)

In [ ]:
dataset_train

In [ ]:
dataset_test

In [ ]:
model_name = 'IlyaGusev/rut5_base_sum_gazeta'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [ ]:
def tokenize(batch):
    tokenized_input = tokenizer(batch['text'], batch['title'], padding='max_length', truncation=True, max_length=max_len_sum)
    # tokenized_label = tokenizer(batch['title'], padding='max_length', truncation=True, max_length=max_len_tl)

    # tokenized_input['labels'] = tokenized_label['input_ids']

    return tokenized_input


# Сформируем токены и удалим лишние столбцы

dataset_train = dataset_train.map(tokenize, batched=True, batch_size=8)
dataset_test = dataset_test.map(tokenize, batched=True, batch_size=8)

dataset_train = dataset_train.remove_columns(['index', 'url', 'title', 'text', 'topic', 'tags', 'date', '__index_level_0__'])
dataset_test = dataset_test.remove_columns(['index', 'url', 'title', 'text', 'topic', 'tags', 'date', '__index_level_0__'])

In [ ]:
dataset_train

In [ ]:
dataset_train.save_to_disk('./')
dataset_test.save_to_disk('./')

In [ ]:
output_dir = 'lenta/output'

training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=1,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    eval_accumulation_steps=1, # Number of eval steps to keep in GPU (the higher, the mor vRAM used)
    prediction_loss_only=True, # If I need co compute only loss and not other metrics, setting this to true will use less RAM
    learning_rate=0.001,
    evaluation_strategy='steps', # Run evaluation every eval_steps,
    # save_strategy='epoch',
    save_steps=500, # How often to save a checkpoint
    save_total_limit=1, # Number of maximum checkpoints to save
    remove_unused_columns=True, # Removes useless columns from the dataset
    # run_name='run_gazeta', # Wandb run name
    logging_steps=500, # How often to log loss to wandb
    eval_steps=500, # How often to run evaluation on the val_set
    logging_first_step=False, # Whether to log also the very first training step to wandb
    load_best_model_at_end=True, # Whether to load the best model found at each evaluation.
    metric_for_best_model='loss', # Use loss to evaluate best model.
    greater_is_better=False, # Best model is the one with the lowest loss, not highest.
    report_to='all',
    optim = 'adamw_torch',
    max_steps=500
)

In [ ]:
gc.collect()

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_test,
    data_collator=data_collator
)

trainer.train()

In [ ]:
trainer.save_model(output_dir + '/model')

In [ ]:
device = 'cuda'

In [ ]:
def title_generate(text_index):

    input_text = test['text'][text_index]

    with torch.no_grad():
        tokenized_text = tokenizer(input_text, truncation=True, padding=True, return_tensors='pt')

        source_ids = tokenized_text['input_ids'].to(device, dtype = torch.long)
        source_mask = tokenized_text['attention_mask'].to(device, dtype = torch.long)

        generated_ids = model.generate(
            input_ids=source_ids,
            attention_mask=source_mask, 
            max_length=20,
            num_beams=10,
            # temperature = 1.3,
            repetition_penalty=1., 
            length_penalty=2, 
            early_stopping=True,
            no_repeat_ngram_size=1
        )

        pred = tokenizer.decode(generated_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
        print(f"PREDICTION: {pred}")

In [ ]:
INX = 7471
print(f"TEXT: {test['text'][INX]}")
print(f"TITLE: {test['title'][INX]}")
title_generate(INX)

In [ ]:
INX = 1869
print(f"TEXT: {test['text'][INX]}")
print(f"TITLE: {test['title'][INX]}")
title_generate(INX)

In [ ]:
INX = 8139
print(f"TEXT: {test['text'][INX]}")
print(f"TITLE: {test['title'][INX]}")
title_generate(INX)